<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-image-classification/notebooks/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像分類（Image Classification）
入力した画像情報を「形」や「模様」、「色」など特徴量に分解し、入力情報から対象物を判定できるように対応付けを行う

## CNN（Convolutional Neural Network）
畳み込みニューラルネットワーク（CNNまたはConvNet）は、畳み込みを使用しているニューラルネットワークの総称

## ResNet
残差ニューラルネットワーク（ResNet）は、ウェイト層が層入力を参照して残差関数を学習する深層学習モデル

## VGG16
ImageNetと呼ばれる大規模画像データセットで学習された16層(畳み込み13層、フル結合3層)からなる畳み込みニューラルネットワーク(CNN)

## Efficient Net
複合スケーリング手法を用いて、ネットワークの幅、深さ、解像度を同時に最適化し、より少ないパラメータ数で高い精度を持つCNN

In [29]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications

# CIFAR-10データセットの読み込み
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
# データの前処理（画素値を0-1の範囲に正規化）
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)  ##
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)  ##

# CNN
if os.path.exists('/content/cnn.keras'):
  cnn_model = tf.keras.models.load_model('/content/cnn.keras')
else:
  # CNNモデルの構築
  print("CNN model not found. Fit CNN model")
  cnn_model = models.Sequential()
  cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  cnn_model.add(layers.MaxPooling2D((2, 2)))
  cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  cnn_model.add(layers.MaxPooling2D((2, 2)))
  cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  cnn_model.add(layers.Flatten())
  cnn_model.add(layers.Dense(64, activation='relu'))
  cnn_model.add(layers.Dense(10, activation='softmax'))

  # モデルのコンパイル
  # optimizer: 最適化アルゴリズム, loss: 損失関数, metrics: 評価指標
  cnn_model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  # モデルの学習
  history = cnn_model.fit(x_train, y_train, batch_size=256, epochs=10,
                     validation_data=(x_test, y_test))

  cnn_model.save('cnn.keras')

# モデルの評価
test_loss, test_acc = cnn_model.evaluate(x_test,  y_test, verbose=2)
print(f'\nCNN: Test accuracy: {test_acc}')


# ResNet
if os.path.exists('/content/resnet.keras'):
  resnet_model = tf.keras.models.load_model('/content/resnet.keras')
else:
  # ResNet50モデルの読み込み（ImageNetで事前学習済み）
  # ImageNet:
  # 物体認識ソフトウェアの研究において、ソフトウェアの最適化や性能の評価等に
  # 用いるために設計された、大規模な画像データセット
  resnet_pre_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

  # ResNet50の出力層をCIFAR-10用に変更
  # 事前学習済みのResNet50モデルに、CIFAR-10のクラス数に対応する全結合層を追加
  resnet_model = models.Sequential()
  resnet_model.add(resnet_pre_model)
  resnet_model.add(layers.Flatten())
  resnet_model.add(layers.Dense(10, activation='softmax'))

  # モデルのコンパイル
  resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # モデルの学習
  # バッチサイズ: モデルが一度に処理するデータの量
  # エポック数: データセット全体を何回繰り返して学習するか
  resnet_model.fit(x_train, y_train, batch_size=256, epochs=10, validation_data=(x_test, y_test))

  resnet_model.save('resnet.keras')

# モデルの評価
loss, accuracy = resnet_model.evaluate(x_test, y_test, verbose=0)
print(f'\nResNet: Test Accuracy: {accuracy}')



# # VGG16
# # VGG16モデルの読み込み（ImageNetで事前学習済み）
# vgg16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# # CIFAR-10データセットの読み込み
# (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# # データの前処理
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# # VGG16の出力層をCIFAR-10用に変更
# model = models.Sequential()
# model.add(vgg16_model)
# model.add(layers.Flatten())
# model.add(layers.Dense(10, activation='softmax'))


# # モデルのコンパイル
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # モデルの学習
# model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# # モデルの評価
# loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
# print(f'Test Accuracy: {accuracy}')


# # Efficient Net
# # EfficientNetB0モデルの読み込み（ImageNetで事前学習済み）
# efficientnet_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# # CIFAR-10データセットの読み込み
# (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# # データの前処理
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# # EfficientNetB0の出力層をCIFAR-10用に変更
# model = models.Sequential()
# model.add(efficientnet_model)
# model.add(layers.GlobalAveragePooling2D())  # GlobalAveragePooling2Dを追加
# model.add(layers.Dense(10, activation='softmax'))

# # モデルのコンパイル
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # モデルの学習
# model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# # モデルの評価
# loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
# print(f'Test Accuracy: {accuracy}')


313/313 - 1s - 4ms/step - accuracy: 0.6695 - loss: 0.9589

CNN: Test accuracy: 0.6694999933242798

ResNet: Test Accuracy: 0.7167999744415283
